---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
import numpy as np

path = "/content/drive/MyDrive/Colab Notebooks/COURSERA - assigment 4/"

feat_importance = ['violation_street_number',
                   'ticket_issued_date',
                   'hearing_date',
                   'fine_amount', 
                   'admin_fee', 
                   'state_fee', 
                   'late_fee', 
                   'discount_amount',
                   'judgment_amount', 
                   "agency_name",
                  #  "inspector_name",
                   "state",
                   "non_us_str_code",
                   "country",
                   "violation_code",
                   "disposition"]

x_test = pd.read_csv(path+"test.csv",
                     usecols=np.append(feat_importance, ["ticket_id", "violator_name"]),
                     encoding="latin-1")
addresess = pd.read_csv(path+"addresses.csv",encoding="latin-1")
latlons = pd.read_csv(path+"latlons.csv", encoding="latin-1")

train = pd.read_csv(path+"train.csv", encoding="latin-1",
                    dtype = {"non_us_str_code": np.object},
                    usecols = np.append(x_test.columns, ["compliance"]))

# train = train[~train["compliance"].isna()]
train.loc[train["compliance"].isna(), "compliance"] = 0

x_train, y_train = train.drop("compliance", axis="columns") , train[["ticket_id","compliance"]]

y_train.set_index("ticket_id", inplace=True)

del train

In [9]:
###### add columns to x_train and x_test
address_latlon = pd.merge(addresess, latlons, on="address")
address_latlon.drop("address", axis="columns", inplace=True)

x_train = pd.merge(x_train, address_latlon, on="ticket_id")
x_test = pd.merge(x_test, address_latlon, on="ticket_id")

del address_latlon
del addresess
del latlons

# #convert to time epoch
x_train["ticket_issued_date"] = pd.to_datetime(x_train["ticket_issued_date"], format="%Y-%m-%d %H:%M:%S")
x_train["hearing_date"] = pd.to_datetime(x_train["hearing_date"], format="%Y-%m-%d %H:%M:%S")
x_test["ticket_issued_date"] = pd.to_datetime(x_test["ticket_issued_date"], format="%Y-%m-%d %H:%M:%S")
x_test["hearing_date"] = pd.to_datetime(x_test["hearing_date"], format="%Y-%m-%d %H:%M:%S")

x_train["ticket_issued_date"] = (x_train["ticket_issued_date"] - pd.to_datetime(0, unit="s")).apply(lambda t: t.total_seconds())
x_train["hearing_date"] = (x_train["hearing_date"] - pd.to_datetime(0, unit="s")).apply(lambda t: t.total_seconds())

x_test["ticket_issued_date"] = (x_test["ticket_issued_date"] - pd.to_datetime(0, unit="s")).apply(lambda t: t.total_seconds())
x_test["hearing_date"] = (x_test["hearing_date"] - pd.to_datetime(0, unit="s")).apply(lambda t: t.total_seconds())

#fill na values
x_train["violator_name"].fillna("no name", inplace=True)
x_test["violator_name"].fillna("no name", inplace=True)

for col in x_train.columns:
  if x_train[col].dtype == "float64":
    if x_train[x_train[col].isna()].size > 0:
      x_train[col].fillna(0, inplace=True)

for col in x_test.columns:
  if x_test[col].dtype == "float64":
    if x_test[x_test[col].isna()].size > 0:
      x_test[col].fillna(0, inplace=True)

# look how many tickets has the violatior
x_train_count = x_train.groupby("violator_name")["ticket_id"].count()
x_test_count = x_test.groupby("violator_name")["ticket_id"].count()

x_train["count"] = x_train.apply(lambda e: x_train_count[e["violator_name"]], axis=1)
x_test["count"] = x_test.apply(lambda e: x_test_count[e["violator_name"]], axis=1)

x_train.drop(columns="violator_name", inplace=True)
x_test.drop(columns="violator_name", inplace=True)

x_train.set_index("ticket_id", inplace=True)
x_test.set_index("ticket_id", inplace=True)

del x_train_count
del x_test_count

In [10]:
# apply get_dummies

dummy_cols = [
              "agency_name",
              # "inspector_name",
              "state",
              "non_us_str_code",
              "country",
              "violation_code",
              "disposition"
]

# for col in dummy_cols:
#   print(f"{col}: {pd.unique(x_train[col]).size}")

concat_train_test_x = pd.concat([x_train, x_test], keys=["x_train", "x_test"])
concat_train_test_x = pd.get_dummies(concat_train_test_x, columns=dummy_cols, dummy_na=True)

x_train = concat_train_test_x[concat_train_test_x.index.isin(["x_train"], level=0)]
x_test = concat_train_test_x[concat_train_test_x.index.isin(["x_test"], level=0)]

x_train = x_train.droplevel(0)
x_test = x_test.droplevel(0)

del concat_train_test_x

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# x_std = StandardScaler().fit_transform(x_train)
# PCA_reductor = PCA(n_components=100).fit(x_std)

# x_pca = PCA_reductor.transform(x_std)
# x_train2, x_test2, y_train2, y_test2 = train_test_split(x_train, y_train)

x_train2, x_test2, y_train2, y_test2 = train_test_split(x_train, y_train)

CLF = GradientBoostingClassifier().fit(x_train2, np.ravel(y_train2))

# CLF = GradientBoostingClassifier().fit(x_train, np.ravel(y_train))

In [13]:
roc_auc_score(y_test2, CLF.predict_proba(x_test2)[:,1])

0.8986744600009048

In [ ]:
def blight_model():
    
    # Your code here
    
    return pd.Series(CLF.predict_proba(x_test)[:,1], index=x_test.index, name="compliance")

In [ ]:
blight_model()

ticket_id
284932    0.07
285362    0.05
285361    0.08
285338    0.16
285346    0.27
          ... 
376496    0.03
376497    0.03
376499    0.16
376500    0.16
369851    0.75
Name: compliance, Length: 61001, dtype: float64